In [4]:
import wget

In [6]:
sld=wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")


  0% [                                                                                ]    0 / 2476
100% [................................................................................] 2476 / 2476

'spacex_launch_dash.csv'

In [7]:
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")


  0% [                                                                                ]    0 / 2110
100% [................................................................................] 2110 / 2110

'spacex_dash_app.py'

In [9]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [10]:
df=pd.read_csv('spacex_launch_dash.csv')
df=df.drop(columns='Unnamed: 0')
df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [11]:
df.shape


(56, 6)

In [12]:
df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [13]:
max_payload= df['Payload Mass (kg)'].max()
min_payload= df['Payload Mass (kg)'].min()


In [14]:
# Create a dash application
app = dash.Dash(__name__)

In [15]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                             dcc.Dropdown(id='site-dropdown',
                                           options=[
                                               {'label': 'All Site','value':'ALL'},
                                               {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                               {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                               {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                               {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                   ],
                                          value='ALL',
                                          placeholder="Select Here a Launch Site",
                                          clearable = False,
                                          searchable = True),
                                html.Br(),
                                
                                 # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                               min=0, max=10000, step=1000,
                                               value=[min_payload, max_payload],
                                               marks={0:'0',
                                                      2500:'2500',
                                                      5000:'5000',
                                                      7500:'7500', 
                                                      10000:'10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

 #TASK 2
 # Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'), 
    Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig1 = px.pie(df, values='class', 
                     names='Launch Site', 
                     title='Success Count for all launch sites')
        return fig1
    else:
        filtered_df=df.loc[df['Launch Site'] == entered_site]
        
        fig2 = px.pie(filtered_df, names='class',title=f"Total Success Launches from site {entered_site}")
        return fig2
        
  #task 4
@app.callback(
     Output(component_id='success-payload-scatter-chart', component_property='figure'),
     [Input(component_id='site-dropdown', component_property='value'),
      Input(component_id='payload-slider', component_property='value')])

def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = df[(df['Payload Mass (kg)']>=payload_slider[0])
                          &(df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot=px.scatter(filtered_data,x="Payload Mass kg", y="class",
                              color="Booster Version Category")
        return scatterplot
    else:
        specific_df = df.loc[df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['Payload Mass (kg)']>= payload_slider[0])
                                   &(df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot =px.scatter(filtered_data, x= "Payload Mass (kg)", y="class", color="Booster Version Category")
        return scatterplot
    

    


        
    
 

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()
                             